# Progetto Ulteriori Attività Formative (UAF): analisi impronte digitali per autenticazione

## Importazione librerie

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import hashlib

ModuleNotFoundError: No module named 'cv2'

## Definizione funzioni per pre-processamento e calcolo della similarità

### Pre-processamento delle immagini

In [3]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Errore nel caricare l'immagine: {image_path}")
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, binary_image = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    return binary_image

### Calcolo similarità bordi

In [ ]:
def compare_edges(edges1, edges2):
    diff = cv2.absdiff(edges1, edges2)
    return np.sum(diff)

### Rilevamento dei bordi

In [ ]:
# Canny Edge Detector
def detect_edges(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

### Estrazione Keypoints

In [ ]:
# Descrittore Oriented Fast end Rotated Brief (ORB)
def extract_keypoints_and_descriptors(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

### Matching tra descrittori

In [ ]:
# BFMatcher
def match_keypoints(descriptors_train, descriptors_test):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors_train, descriptors_test)
    matches = sorted(matches, key = lambda x: x.distance)
    return matches

### Calcolo e confronto Hash delle immagini

In [ ]:
def calculate_image_hash(image_path):
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
        return hashlib.md5(image_bytes).hexdigest()

def compare_hashes(hash1, hash2):
    return hash1 == hash2

## Categorizzazione immagini

In [ ]:
def categorize_images(image_path_train, image_path_test):
    # Preprocessa le immagini
    processed_image_train = preprocess_image(image_path_train)
    processed_image_test = preprocess_image(image_path_test)

    # Calcolare i bordi per due immagini (in questo caso, le immagini preprocessate train e test)
    edges_image_train = detect_edges(processed_image_train)
    edges_image_test = detect_edges(processed_image_test)

    # Calcolare la similarità dei bordi tra le due immagini
    edge_similarity = compare_edges(edges_image_train, edges_image_test)

    # Estrai keypoints e descrittori per le due immagini
    keypoints_train, descriptors_train = extract_keypoints_and_descriptors(processed_image_train)
    keypoints_test, descriptors_test = extract_keypoints_and_descriptors(processed_image_test)

    # Fai il matching tra i descrittori delle due immagini
    matches = match_keypoints(descriptors_train, descriptors_test)

    # Calcola la somiglianza in base al numero di match
    similarity_keypoints = len(matches)

    # Calcola gli hash delle due immagini
    hash_train = calculate_image_hash(image_path_train)
    hash_test = calculate_image_hash(image_path_test)

    # Confronta gli hash
    hash_similarity = compare_hashes(hash_train, hash_test)

    # Determina la categoria dell'immagine
    if hash_similarity:
        category = "Uguali (hash identici)"
    else:
        category = "Diverse (bassi match e differenze nei bordi)"
    
    return category, keypoints_train, keypoints_test, matches, edges_image_train, edges_image_test, similarity_keypoints, edge_similarity

image_files_train = [f for f in os.listdir(train_folder) if f.endswith('.tif')]
image_files_test = [f for f in os.listdir(test_folder) if f.endswith('.tif')]

# Variabili per memorizzare una coppia per ogni caso
equal_image_pair = None
different_image_pair = None

# Cerca e categorizza le immagini
for image_file_train in image_files_train:
    image_path_train = os.path.join(train_folder, image_file_train)
    for image_file_test in image_files_test:
        image_path_test = os.path.join(test_folder, image_file_test)

        # Categorizza la coppia
        category, keypoints_train, keypoints_test, matches, edges_image_train, edges_image_test, similarity_keypoints, edge_similarity = categorize_images(image_path_train, image_path_test)

        # Assegna la coppia al relativo caso
        if category == "Uguali (hash identici)" and equal_image_pair is None:
            equal_image_pair = (image_path_train, image_path_test, keypoints_train, keypoints_test, matches, edges_image_train, edges_image_test)
        elif category == "Diverse (bassi match e differenze nei bordi)" and different_image_pair is None:
            different_image_pair = (image_path_train, image_path_test, keypoints_train, keypoints_test, matches, edges_image_train, edges_image_test)

        # Break if we have both cases
        if equal_image_pair and different_image_pair:
            break
    if equal_image_pair and different_image_pair:
        break

## Visualizzazione immagini

In [ ]:
def display_results(image_pair, title):
    image_path_train, image_path_test, keypoints_train, keypoints_test, matches, edges_image_train, edges_image_test = image_pair

    # Carica le immagini originali per la visualizzazione
    img_train = cv2.imread(image_path_train)
    img_test = cv2.imread(image_path_test)

    # Visualizza i bordi
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(edges_image_train, cmap='gray')
    plt.title('Bordi Immagine Train')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(edges_image_test, cmap='gray')
    plt.title('Bordi Immagine Test')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    diff_edges = cv2.absdiff(edges_image_train, edges_image_test)
    plt.imshow(diff_edges, cmap='hot')
    plt.title(f'Differenza Bordi')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Visualizza i keypoints ORB
    img_with_keypoints_train = cv2.drawKeypoints(img_train, keypoints_train, None, color=(0, 255, 0))
    img_with_keypoints_test = cv2.drawKeypoints(img_test, keypoints_test, None, color=(0, 255, 0))

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img_with_keypoints_train)
    plt.title('Keypoints Immagine Train')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img_with_keypoints_test)
    plt.title('Keypoints Immagine Test')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Visualizza i match tra i keypoints
    matched_image = cv2.drawMatches(img_train, keypoints_train, img_test, keypoints_test, matches[:20], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(10, 6))
    plt.imshow(matched_image)
    plt.title(f"Numero di Match tra Keypoints: {len(matches)}")
    plt.axis('off')
    plt.show()

### Caso 1: coppia di immagini uguali

In [ ]:
# Visualizza i risultati per ogni caso
print("Immagini Uguali")
display_results(equal_image_pair, "Immagini Uguali")

### Caso 2: coppia di immagini diverse

In [ ]:
print("Immagini Diverse)")
display_results(different_image_pair, "Immagini Diverse")